# Holm-Hone equation

In [ ]:
include("setup.jl")

# Periodic boundary conditions

In [ ]:
function relaxation_functional(γ, unew, uold, param)
    @unpack D, D2a, D4a, tmp1, tmp2, tmp3 = param
    
    @. tmp3 = (1-γ)*uold + γ*unew
    mul!(tmp2, D2a, tmp3)
    mul!(tmp1, D4a, tmp3)
    @. tmp1 = tmp3 * (4*tmp3 - 5*tmp2 + tmp1)
    energy = integrate(tmp1, D)
end
function relaxation_functional(u, param)
    @unpack D, D2a, D4a, tmp1, tmp2 = param
    
    mul!(tmp2, D2a, u)
    mul!(tmp1, D4a, u)
    @. tmp1 = u * (4*u - 5*tmp2 + tmp1)
    energy = integrate(tmp1, D)
end


function save_func_hh_periodic(u, t, integrator)
    @unpack D, D2a, D4a, tmp1, tmp2, tmp3, usol = integrator.p
    print(".")
    
    mass = integrate(u, D)
    
    mul!(tmp2, D2a, u)
    mul!(tmp3, D4a, u)
    @. tmp1 = 4*u - 5*tmp2 + tmp3
    linear = integrate(tmp1, D)
    
    @. tmp1 = u * tmp1
    quadratic = integrate(tmp1, D)
    
    x = grid(D)
    tmp1 .= ( usol.(t, x, x[1], -x[1]) .- u ).^2
    error_u = integrate(tmp1, D) |> sqrt
    
    SVector(mass, linear, quadratic, error_u)
end


function hh_periodic!(du, u, param, t)
    @unpack D, inv4m5D2pD4, D2b, D4b, tmp1, tmp2, tmp3 = param
    
    # conservative semidiscretization
    mul!(tmp1, D2b, u)
    mul!(tmp2, D4b, u)
    @. tmp1 = -(4 * u - 5 * tmp1 + tmp2)
    @. tmp2 = u * tmp1
    mul!(tmp3, D, tmp2)
    mul!(tmp2, D, u)
    @. tmp3 = tmp3 + tmp2 * tmp1
    ldiv!(du, inv4m5D2pD4, tmp3)
    
    nothing
end

function solve_ode_hh_periodic(usol, D, D2a, D2b, D4a, D4b, tspan, alg, tol, dt, adaptive)
    # lu instead of fatorize because of https://github.com/JuliaLang/julia/issues/30084
    inv4m5D2pD4 = isa(D2a, AbstractMatrix) ? lu(4*I - 5*D2a + D4a) : 4*I - 5*D2a + D4a
    
    x = grid(D)
    u0 = usol.(tspan[1], x, x[1], -x[1])
    tmp1 = similar(u0); tmp2 = similar(tmp1); tmp3 = similar(tmp1)
    param = (D=D, D2a=D2a, D2b=D2b, D4a=D4a, D4b=D4b, inv4m5D2pD4=inv4m5D2pD4, 
             tmp1=tmp1, tmp2=tmp2, tmp3=tmp3, usol=usol)

    ode = ODEProblem(hh_periodic!, u0, tspan, param)
    
    saveat = range(tspan..., length=100)
    saved_values_baseline = SavedValues(eltype(D), SVector{4,eltype(D)})
    saving_baseline = SavingCallback(save_func_hh_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{4,eltype(D)})
    saving_relaxation = SavingCallback(save_func_hh_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!, save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_relaxation, tstops=saveat)
    flush(stdout)
    sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_baseline, tstops=saveat)
    flush(stdout)

    unum_baseline   = sol_baseline[end]
    unum_relaxation = sol_relaxation[end]
    uana = usol.(tspan[end], x, x[1], -x[1])
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, unum_baseline - uana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, unum_relaxation - uana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, unum_baseline - unum_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
    plt.plot(x, uana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    mass_baseline        = map(x->x[1], saved_values_baseline.saveval)
    mass_relaxation      = map(x->x[1], saved_values_relaxation.saveval)
    linear_baseline      = map(x->x[2], saved_values_baseline.saveval)
    linear_relaxation    = map(x->x[2], saved_values_relaxation.saveval)
    quadratic_baseline   = map(x->x[3], saved_values_baseline.saveval)
    quadratic_relaxation = map(x->x[3], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   mass_baseline   .- mass_baseline[1], 
        label=L"$\int u$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, mass_relaxation .- mass_relaxation[1], 
        label=L"$\int u$ (conservative)", color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   linear_baseline   .- linear_baseline[1], 
        label=L"$\int (4 - 5 \partial_x^2 + \partial_x^4) u$ (non-conservative)", color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, linear_relaxation .- linear_relaxation[1], 
        label=L"$\int (4 - 5 \partial_x^2 + \partial_x^4) u$ (conservative)", color="#56B4E9", linestyle="--")
    plt.plot(t_baseline,   quadratic_baseline   .- quadratic_baseline[1], 
        label=L"$\int u (4 - 5 \partial_x^2 + \partial_x^4) u$ (non-conservative)", color="#E69F00", linestyle=":")
    plt.plot(t_relaxation, quadratic_relaxation .- quadratic_relaxation[1], 
        label=L"$\int u (4 - 5 \partial_x^2 + \partial_x^4) u$ (conservative)", color="#56B4E9", linestyle=":")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_u_baseline   = map(x->x[end], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[end], saved_values_relaxation.saveval)

    fig_error, ax = plt.subplots(1, 1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline,   error_u_baseline, 
        label=L"Error of $u$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_u_relaxation, 
        label=L"Error of $u$ (conservative)", color="#56B4E9", linestyle="-")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    (; sol_relaxation, sol_baseline, saved_values_baseline, saved_values_relaxation, 
        fig_u, fig_invariants, fig_error)
end


In [ ]:
# traveling wave solution obtained numerically
c, data = open("hh_traveling_wave_init_c12_ah01_l40_N65536.txt", "r") do io
    line = readline(io)
    line = readline(io)
    line = readline(io)
    c = parse(Float64, line[8:end])
    data = readdlm(io, comments=true)
    c, data
end
x = range(data[1,1], data[end,1], length=size(data,1))
xmin = x[1]; xmax = x[end]
u0 = data[:, 2]
u0itp = CubicSplineInterpolation((x,), u0, extrapolation_bc=Periodic())
get_c() = c
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)
end

In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show usol(0., xmin, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^9
@show dt = 0.5 * (xmax - xmin) / (N * get_c())

@show tspan = (0.0, (xmax-xmin)/(3*get_c()) + 100*(xmax-xmin)/get_c())
flush(stdout)

tol = 1.0e-8
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)
D2a = D2b = D^2; D4a = D4b = D^4

results = solve_ode_hh_periodic(usol, D, D2a, D2b, D4a, D4b, tspan, Tsit5(), tol, dt, adaptive);

In [ ]:
ax = results.fig_error.axes[1]
t = [1.0e2, 6.0e3]
ax.plot(t, 1.0e-9 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (1.0e3, 8.0e-3), color="gray")
ax.plot(t, 5.0e-10 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (1.0e3, 2.0e-7), color="gray")

results.fig_error.savefig(joinpath(dirname(@__DIR__), "figures", "hh_error.pdf"), 
    bbox_inches="tight")